## Define the Objective of the Analysis

...

Analisi del sentiment relativo al dibattito sullo _Smart Working_ in Italia e individuazione di eventuali 'influencer' sul tema.

## Collect Data

In [1]:
# imports
import numpy as np
import pandas as pd
import tweepy
tweepy.__version__

import re
import string
import itertools
from collections import Counter
from datetime import datetime
from numpy.core.multiarray import result_type
import time

In [2]:
# insert the keys here
consumer_key = 'VPzjkqKl2y1uSTJQvnVqS9e1X' 
consumer_secret = 'STG2IzVMf65vPGeOvBQyzdeoKBExAr5sIkhOaBeDe2fnIN14vY'
access_token = '1508409949835214853-HIyZJ3oT32TijKsdNDhGFZEEQTWwau'
access_token_secret = 'uLcs9hUYmLdocxkaSfXo69Gii46TISu5qZj5F6f6fBfnW'

#### Tweets Download

The next step is creating an OAuthHandler instance. We pass our consumer key and access token which we defined above.

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

Next, we pass the OAuthHandler instance into the API method.

In [ ]:
api = tweepy.API(auth)

In [4]:
api = tweepy.API(auth, wait_on_rate_limit=True)

Tweets that contain a specific hashtag

In [ ]:
from tqdm.notebook import tqdm

import time
hashtag = '("#smartworking" OR "#remotework" OR "#lavoroagile")'

list_tweets = []

for i in range(10):
    for tweet in tweepy.Cursor(api.search_tweets, q=hashtag, count=50, lang='it').items(850):
      print('entering')
      full_text = api.get_status(tweet.id, tweet_mode='extended')._json['full_text']
      print(tweet.id)
      list_tweets.append([tweet.created_at, tweet.id, full_text, tweet.favorite_count, tweet.retweet_count, tweet.user.screen_name,
      tweet.user.location, tweet.retweeted, tweet.entities['user_mentions'], tweet.entities['hashtags']])
    for j in range(60):
        time.sleep(1)

# items is the maximum number of tweets to download.
# count is the number of tweets to return per page, up to a maximum of 100.

In [5]:
from tqdm.notebook import tqdm

import time
hashtag = '("#smartworking" OR "#remotework" OR "#lavoroagile")'

list_tweets = []

for i in range(10):
    for tweet in tweepy.Cursor(api.search_tweets, q=hashtag, count=100, lang='it').items(10000):
      print('entering')
      full_text = api.get_status(tweet.id, tweet_mode='extended')._json['full_text']
      print(tweet.id)
      list_tweets.append([tweet.created_at, tweet.id, full_text, tweet.favorite_count, tweet.retweet_count, tweet.user.screen_name,
      tweet.user.location, tweet.retweeted, tweet.entities['user_mentions'], tweet.entities['hashtags']])
    for j in range(60):
        time.sleep(1)

# items is the maximum number of tweets to download.
# count is the number of tweets to return per page, up to a maximum of 100.

entering
1609198624671469570
entering
1609193893228150787
entering
1609192822271270914
entering
1609188720170647559
entering
1609186120457240581
entering
1609178943004745730
entering
1609174559848013826
entering
1609167751712587776
entering
1609166621721133062
entering
1609152087874588674
entering
1609143321082880001
entering
1609142147751485443
entering
1609139563749953537
entering
1609138643452526593
entering
1609136259158474753
entering
1609135952437383169
entering
1609135637373894658
entering
1609132156290203651
entering
1609130798166204416
entering
1609129648532643841
entering
1609129409696632836
entering
1609128976215117828
entering
1609128270305296384
entering
1609128030936662016
entering
1609126076105781248
entering
1609122191978926089
entering
1609118337346457600
entering
1609117188056571904
entering
1609112345607032834
entering
1609112339231477771
entering
1609106278282039298
entering
1609098673891278851
entering
1609094875705741317
entering
1609094017249333249
entering
16090

Rate limit reached. Sleeping for: 650


entering
1605913032823029764
entering
1605912836332568576
entering
1605910904582787073
entering
1605910535626784771
entering
1605909957077073920
entering
1605908311009787905
entering
1605907981731827713
entering
1605907644694274050
entering
1605907629145878529
entering
1605907479182725125
entering
1605907345686507523
entering
1605907117667254273
entering
1605903563867279360
entering
1605901772484132868
entering
1605901624068620288
entering
1605899661759303681
entering
1605899501213999104
entering
1605899107549184006
entering
1605897065703378946
entering
1605896863714099200
entering
1605893777809457153
entering
1605893749615255553
entering
1605893257296105473
entering
1605892917637283843
entering
1605892830613872640
entering
1605892582063710208
entering
1605892434923225090
entering
1605891228859514880
entering
1605890372651036672
entering
1605888374442643458
entering
1605888095177564160
entering
1605887529982595072
entering
1605887174406246400
entering
1605887027924373505
entering
16058

Rate limit reached. Sleeping for: 591


1607441620219334656
entering
1607421732364632065
entering
1607413857772883969
entering
1607408810796224513
entering
1607408676658204675
entering
1607408626414534657
entering
1607408556331917315
entering
1607408524413513728
entering
1607408475554074624
entering
1607408442628612098
entering
1607408204027232257
entering
1607407469340545025
entering
1607404953743630340
entering
1607404262664855552
entering
1607404044871684097
entering
1607367595732402177
entering
1607347529712025604
entering
1607345836693897216
entering
1607345786970333187
entering
1607345633899200514
entering
1607336174539997184
entering
1607329097801244673
entering
1607320877657473025
entering
1607320074196492289
entering
1607313894556459009
entering
1607312338386051073
entering
1607311841453015046
entering
1607311528339902464
entering
1607307076157734913
entering
1607306973199859717
entering
1607306644517527552
entering
1607306441156710400
entering
1607305987194494976
entering
1607304641435279362
entering
16073034175879

...

Rate limit reached. Sleeping for: 672


1605883754630844417
entering
1605883473759289344
entering
1605882977740881922
entering
1605882883289206786
entering
1605882699650039808
entering
1605881121597804544
entering
1605881008519159808
entering
1605880894341857281
entering
1605880535024279552
entering
1605879798621999105
entering
1605877993640902660
entering
1605877138061725699
entering
1605876422328799233
entering
1605875194547036161
entering
1605874259368329216
entering
1605872817249058817
entering
1605870946430197760
entering
1605870187747479552
entering
1605870185474408448
entering
1605870124291850241
entering
1605868928223395840
entering
1605868812544393218
entering
1605868799630053376
entering
1605868700875333632
entering
1605868528984367105
entering
1605868418967683072
entering
1605867964154028035
entering
1605867887150997506
entering
1605866874553733121
entering
1605866853191995394
entering
1605866793788227585
entering
1605866410957082624
entering
1605865823096143872
entering
1605865226951397377
entering
16058641367299

Rate limit reached. Sleeping for: 669


1605679292049940499
entering
1605679133400346624
entering
1605679093848035329
entering
1605678728884822016
entering
1605677751503142913
entering
1605677733815672836
entering
1605677715318792221
entering
1605676328723677184
entering
1605675903970746368
entering
1605675899088560128
entering
1605675776271081493
entering
1605675685632172055
entering
1605675636579786752
entering
1605675575120605184
entering
1605675485215744005
entering
1605675409000919040
entering
1605675385844142082
entering
1605675338159095808
entering
1605675287139622913
entering
1605675187571019777
entering
1605675141110849557
entering
1605675126585888768
entering
1605675104570073111
entering
1605675066833846272
entering
1605675058348756993
entering
1605674922910572567
entering
1605674905084710912
entering
1605674835048247296
entering
1605674805742559232
entering
1605674793591791628
entering
1605674727015518209
entering
1605674718006239265
entering
1605674653233487872
entering
1605674223443156992
entering
16056741914742

KeyboardInterrupt: 

In [6]:
tweets = pd.DataFrame(list_tweets, columns=['date','id','text','like','n_rt','author','location','retweeted','user_mentions','hastags'])
tweets.head()

,date,id,text,like,n_rt,author,location,retweeted,user_mentions,hastags
0,2022-12-31 14:43:44+00:00,1609198624671469570,RT @aleale2780: @carettamc11 @FratellidItalia ...,0,11,alessiobar4,,False,"[{'screen_name': 'aleale2780', 'name': 'Aleale...","[{'text': 'opzionedonna', 'indices': [113, 126]}]"
1,2022-12-31 14:24:56+00:00,1609193893228150787,RT @aleale2780: @carettamc11 @FratellidItalia ...,0,11,martin_marzia,,False,"[{'screen_name': 'aleale2780', 'name': 'Aleale...","[{'text': 'opzionedonna', 'indices': [113, 126]}]"
2,2022-12-31 14:20:40+00:00,1609192822271270914,RT @marcoz984: Ripetete con me: lo #smartworki...,0,7,Ernesto23710724,,False,"[{'screen_name': 'marcoz984', 'name': 'marco d...","[{'text': 'smartworking', 'indices': [35, 48]}]"
3,2022-12-31 14:04:22+00:00,1609188720170647559,RT @IlConteIT: Per contrastare una nuova possi...,0,4,antomariateres1,,False,"[{'screen_name': 'IlConteIT', 'name': 'ConCont...","[{'text': 'Covid', 'indices': [63, 69]}, {'tex..."
4,2022-12-31 13:54:03+00:00,1609186120457240581,RT @dukana2: Toglietevi subito quelle mascheri...,0,11,dukana2,,False,"[{'screen_name': 'dukana2', 'name': 'dukana', ...","[{'text': 'Smartworking', 'indices': [72, 85]}..."


In [7]:
len(list_tweets)

12946

In [8]:
# Turn list_tweet into a DataFrame changing column names
tweets = pd.DataFrame(list_tweets, columns=['date','id','text','like','n_rt','author','location','retweeted','user_mentions','hastags'])
tweets.to_csv('../data/SW2.csv')

In [ ]:
tweets_loaded = pd.read_csv('../data/SW.csv')
#tweets_loaded = tweets.drop('Unnamed: 0', axis=1)
tweets_loaded.head()

In [ ]:
tweets.shape

In [ ]:
tweets.columns

#### Data Pre-Processing

In [ ]:
# Remove duplicates
tweets.drop_duplicates(subset ="id", inplace = True)
tweets.reset_index(drop = True, inplace = True)
tweets.shape

In [ ]:
# Change date format
day = tweets['date'].dt.day
month = tweets['date'].dt.month
year = tweets['date'].dt.year

date = year.astype(str) + month.astype(str).str.zfill(2) + day.astype(str).str.zfill(2)
date = pd.to_datetime(date, format='%Y%m%d')
tweets.drop(columns = ['date'], inplace = True)
tweets['date'] = date

# Reorder columns
cols = tweets.columns.tolist()
cols = cols[-1:] + cols[:-1]
tweets = tweets[cols].copy()

print('Tweet per day:')
print()
print(tweets.groupby('date').count()['id'])
# print()
# print()
# print('Minimum Tweet ID per day:')
# print()
# print(tweets.groupby('date').min('id')['id'])
# print()
# print()
# print('Maximum Tweet ID per day:')
# print()
# print(tweets.groupby('date').max('id')['id'])

In [ ]:
# Create a DataFrame with the authors of the tweets and their respective frequency
freq_authors = tweets['author'].value_counts()
freq_authors.head()

In [ ]:
# Extract all hashtags from the full text
tweets['hashtags_list'] = tweets['text'].apply(lambda x: re.findall(r"#(\w+)", x))

# Extract all mentions from the full text
tweets['mentions'] = tweets['text'].apply(lambda x: re.findall(r"@(\w+)", x))

# \w matches any single letter, number or underscore (same as [a zA Z0 9_])

#### Data Cleaning

NLTK - Natural Language ToolKit is a platform for building Python programs to work with human language data. It provides easy to use interfaces to over 50 corpora and lexical resources, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning.

In [ ]:
import nltk
from nltk import FreqDist
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

Text Pre-Processing

In [ ]:
def preprocess_data(data):
    # remove numbers and turning words into lower case
    data = data.astype(str).str.replace('\d+','')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer = TweetTokenizer()
    
    # token lemmatization (ex. goes --> go)
    def lemmatize_text(text):
        return[(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    
    # remove punctuation
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

In [ ]:
# apply preprocess_data function
pre_tweets = preprocess_data(tweets['text'])
tweets['text_proc'] = pre_tweets

# delete italian stopwords
stop_words = set(stopwords.words('italian'))
tweets['text_proc'] = tweets['text_proc'].apply(lambda x: [item for item in x if item not in stop_words])

## Analysis

### Social Content Analysis

#### _Sentiment Analysis_

When dealing with social media text, we usually want to identify urls, hashtags, smileys as separate objects and do not tokenize it to individual characters.

VADER

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

In [ ]:
sent_analyzer = SentimentIntensityAnalyzer()

In [ ]:
tweets[['text', 'text_proc']]

In [ ]:
tweets['scores'] = tweets['text'].apply(lambda Tweet:sent_analyzer.polarity_scores(Tweet))
tweets['compound'] = tweets['scores'].apply(lambda score_dict:score_dict['compound'])

tweets.head()

FEEL-IT: Emotion and Sentiment Classification for the Italian Language.

https://towardsdatascience.com/sentiment-analysis-and-emotion-recognition-in-italian-using-bert-92f5c8fe8a2

In [ ]:
from feel_it import EmotionClassifier, SentimentClassifier

sentiment_classifier = SentimentClassifier()
emotion_classifier = EmotionClassifier()

The feel-it-italian-sentiment model performs sentiment analysis on Italian. We fine-tuned the UmBERTo model on our new dataset (i.e., FEEL-IT) obtaining state-of-the-art performances on different benchmark corpora.

In [ ]:
print(sentiment_classifier.predict(tweets["text"].values.tolist()))

In [ ]:
tweets_sentiment = tweets.copy()

In [ ]:
text = tweets_sentiment['text']
li_sent = []
for i in range(0, text.shape[0]):
  sent = sentiment_classifier.predict([text[i]])
  li_sent.append(sent)
  if i % 5000 == 0:
    print('Riga',i,'su',text.shape[0])

tweets_sentiment['sentiment_BERT'] = [item for sublist in li_sent for item in sublist]
     
positive = []
negative = []

In [ ]:
positive = []
negative = []
ratio = []

for line in tweets_sentiment.values:
  sent = line[15]

  if sent == 'negative':
    positive.append(0)
    negative.append(1)
    ratio.append(-1)
  else:
    positive.append(1)
    negative.append(0)
    ratio.append(1)

In [ ]:
tweets_sentiment['positive'] = positive
tweets_sentiment['negative'] = negative
tweets_sentiment['ratio'] = ratio

In [ ]:
tweets_sentiment.head()

In [ ]:
tweets_sentiment.to_csv('../data/SW_sentiment.csv')

#### _Emotion Analysis_

Recognizing emotions in text is fundamental to get a better sense of how people are talking about something. People can talk about a new event, but positive/negative labels might not be enough.

The feel-it-italian-emotion model performs emotion classification (joy, fear, anger, sadness) on Italian. We fine-tuned the UmBERTo model on our new dataset (i.e., FEEL-IT) obtaining state-of-the-art performances on different benchmark corpora.

In [ ]:
print(emotion_classifier.predict(tweets["text"].values.tolist()))

In [ ]:
tweets_emotion = tweets.copy()

In [ ]:
text = tweets_emotion['text']
li_emotion = []
for i in range(0, text.shape[0]):
  emotion = emotion_classifier.predict([text[i]])
  li_emotion.append(emotion)
  if i % 5000 == 0:
    print('Riga',i,'su',text.shape[0])

tweets_emotion['emotion_BERT'] = [item for sublist in li_emotion for item in sublist]
     
anger = []
joy = []
fear = []
sadness = []

In [ ]:
anger = []
joy = []
fear = []
sadness = []

for line in tweets_emotion.values:
  emotion = line[15]

  if emotion == 'anger':
    anger.append(1)
    joy.append(0)
    fear.append(0)
    sadness.append(0)
  elif emotion == 'joy':
    anger.append(0)
    joy.append(1)
    fear.append(0)
    sadness.append(0)
  elif emotion == 'fear':
    anger.append(0)
    joy.append(0)
    fear.append(1)
    sadness.append(0)
  else:
    anger.append(0)
    joy.append(0)
    fear.append(0)
    sadness.append(1)

In [ ]:
tweets_emotion['anger'] = anger
tweets_emotion['joy'] = joy
tweets_emotion['fear'] = fear
tweets_emotion['sadness'] = sadness

In [ ]:
tweets_emotion.head()

In [ ]:
tweets_emotion.to_csv('../data/SW_emotion.csv')

---

#### Social Network Analysis

Measures of Centrality

Community Detection

## Visualization